In [1]:
#This is our INFO-371 project

In [4]:
#Imports
import os
import pandas as pd
import pathlib
import pyarrow as pa

In [18]:
input_data_dir = os.path.join(os.path.realpath(pathlib.Path().cwd()), "input_data")
train_series = pd.read_parquet(os.path.join(input_data_dir, "train_series.parquet"))
train_events = pd.read_csv(os.path.join(input_data_dir, "train_events.csv"))